In [ ]:
! conda install glib=2.51.0 -y
! pip install kaggle
! pip install matplotlib
! pip install tensorflow_datasets

In [ ]:
import glob
import math
import os
import random
import subprocess

from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPool2D, Softmax
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [ ]:
# Constants
IMAGE_SIZE = (100, 100)
VAL_SIZE = 0.2
BATCH_SIZE = 16
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# Place kaggle token
home = Path.home()
if not home.joinpath(".kaggle").exists():
    os.mkdir(str(home.joinpath(".kaggle")))  # Make /root/.kaggle
if home.joinpath("kaggle.json").exists():
  os.rename(str(home.joinpath("../content/kaggle.json")), str(home.joinpath(".kaggle/kaggle.json")))  # Move /content/kaggle.json to /root/.kaggle/

In [ ]:
# Install dataset
if not 'data_dir' in globals():
  download_manager = tfds.download.DownloadManager(
      download_dir=str(home.joinpath(".tensorflow/datasets")),
      force_extraction=True  # Unzip the dataset
  )
  data_dir = download_manager.download_kaggle_data("grassknoted/asl-alphabet").joinpath("asl_alphabet_train/asl_alphabet_train")

In [ ]:
# Preprocess images
def process_image(image):
  image = cv2.Canny(image, 80, 90)
  image = cv2.dilate(image, None)
  image = cv2.erode(image, None)
  image = cv2.resize(image, IMAGE_SIZE)
  return image

classes_folders = sorted(glob.glob(str(data_dir)+"/*"))
images, labels = [], []
class_to_index_map = {}
index_to_class_map = {}
DROP_RATE = 0.3
for folder in classes_folders:  # /A, /del, etc
    class_name = tf.strings.split(folder, "/")[-1].ref()
    class_index = len(index_to_class_map)
    index_to_class_map[class_index] = class_name
    class_to_index_map[class_name] = class_index
    for filename in glob.glob(folder+"/*.jpg"):
        image = cv2.imread(filename, 0)
        image = process_image(image)
        image = numpy.array(image)
        image = tf.convert_to_tensor(image, dtype=tf.float32)
        label = tf.convert_to_tensor(class_index, dtype=tf.int32)
        images.append(image)
        labels.append(label)

In [ ]:
# Prepare dataset
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.shuffle(len(labels))

train_ds = dataset.skip(math.floor(VAL_SIZE * len(dataset)))
val_ds = dataset.take(math.floor(VAL_SIZE * len(dataset)))

train_ds = train_ds.batch(BATCH_SIZE).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# View some images
plt.figure(figsize=(10, 10))
for _images, _labels in train_ds.take(3):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(tf.squeeze(_images[i]).numpy().astype("uint8"), cmap="gray")
    plt.axis("off")

In [ ]:
# Create the model
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=IMAGE_SIZE + (1,), activation='relu'))  # 64 filters with size 3x3 over a 64x64 image of 1 channel
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D())  # Defaults to (2,2)
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPool2D())  # Defaults to (2,2)
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(Dropout(0.2))  # Drop half the time
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(29, activation='relu'))  # 26 alphabets + del, space, nothing
model.add(Softmax())  # Convert to probabilities

print(model.summary())

In [ ]:
# Compile and train the model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
history = model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=[early_stopping])

In [ ]:
# Evaluate model
plt.plot(history.history.get("accuracy"), label="accuracy")
plt.plot(history.history.get("val_accuracy"), label="val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim([0, 1])
plt.legend(loc="lower right")
plt.show()

In [ ]:
model.save("Hand_Gesture_Model.h5)

In [ ]:
print(class_to_index_map)